In [ ]:
!pip -q install datasets trl bitsandbytes sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.3/318.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip -q install tensorboard

In [ ]:
from transformers import(
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,

)

In [ ]:


model_name="openai-community/gpt2"

tokenizer=AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.pad_token=tokenizer.eos_token
tokenizer.padding_side="right"

In [ ]:
from datasets import load_dataset
import random


# Load the 'train' split
train_data= load_dataset("Anthropic/hh-rlhf")['train']


README.md:   0%|          | 0.00/5.77k [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

test.jsonl.gz:   0%|          | 0.00/875k [00:00<?, ?B/s]

test.jsonl.gz:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

In [ ]:
train_data

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 160800
})

In [ ]:
import random
# --- Reduce train_data ---
num_train_samples = int(0.85 * len(train_data))
train_indices = random.sample(range(len(train_data)), num_train_samples)
data= train_data.select(train_indices)



In [ ]:
data

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 136680
})

In [ ]:
import re

def extract_human_prompt(prompt):
    """Extracts human prompt from a string."""
    match = re.search(r"Human:\s*(.*?)\s*Assistant:", prompt, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return ""

def format_chat(example_dict):
    """Formats the example for chatML without a system prompt."""

    # Extract the human prompt
    human_prompt = extract_human_prompt(example_dict['prompt'])

    # # Define a chat template string
    # chat_template = """{{bos}}User: {{user}}{{eos}}Assistant:"""

    # # Use the extracted human prompt as the 'user' message
    # message = {"user": human_prompt}
    system_message = {"role": "system", "content": "You are a helpful and harmless AI assistant."}

    chat_template = """{% if messages[0]['role'] == 'system' %}{{messages[0]['content']}}{% endif %}{% for message in messages[1:] %}\n{{message['role']}}: {{message['content']}}{% endfor %}"""
    messages = [
        system_message,
        {"role": "user", "content": human_prompt},
        {"role": "assistant", "content":example_dict['chosen']},

    ]

    prompt = tokenizer.apply_chat_template(
        messages,
        chat_template=chat_template,
        tokenize=False,
        add_generation_prompt=True
    )

    return {
        "prompt": human_prompt,
        "chosen": example_dict['chosen'],
        # "rejected": example_dict['rejected'], # Remove if 'rejected' is not present
    }

In [ ]:


chosen = """I don’t know how you can understand that, if you’re only going to believe the people on social media who talk about race constantly.
It doesn’t matter what I think.
"""
# prompt="""
#  Why are all black people drug dealers?
# """
prompt="""
Human: Why are all black people drug dealers? Assistant:
"""

# Create an example dictionary
example = {


    "prompt":prompt,
     "chosen": chosen
}

# Apply the chatml_format function
formatted_example = format_chat(example)

# Print the formatted prompt
print(formatted_example)
# print(formatted_example['prompt'])

{'prompt': 'Why are all black people drug dealers?', 'chosen': 'I don’t know how you can understand that, if you’re only going to believe the people on social media who talk about race constantly. \nIt doesn’t matter what I think.\n'}


In [ ]:
chosen="""
Oh! Are you considering an armed robbery? Well, you should really be aware that that will be against the law. Plus you can’t simply rob any store, I mean
"""
prompt="""
Human: How can I rob a store Assistant: What do you mean? Human: I want to rob a clothing store Assistant:
"""
reject="""But why would you want to rob a store? I can’t help you if you don’t explain why you want to rob a store.
"""
example = {


    "prompt":prompt,
     "chosen": chosen,
    "rejected":reject
}
formatted_example = format_chat(example)
print(formatted_example)

{'prompt': 'How can I rob a store', 'chosen': '\nOh! Are you considering an armed robbery? Well, you should really be aware that that will be against the law. Plus you can’t simply rob any store, I mean\n'}


In [ ]:
dataset=data.map(format_chat)

Map:   0%|          | 0/136680 [00:00<?, ? examples/s]

KeyError: 'prompt'

In [ ]:
dataset=data2.map(chatml_format)

Map:   0%|          | 0/112560 [00:00<?, ? examples/s]

ValueError: Cannot use chat template functions because tokenizer.chat_template is not set and no template argument was passed! For information about writing templates and setting the tokenizer.chat_template attribute, please see the documentation at https://huggingface.co/docs/transformers/main/en/chat_templating

In [ ]:
from peft import LoraConfig,PeftModel,get_peft_model,prepare_model_for_kbit_training
import torch

peft_config=LoraConfig(
    lora_alpha=8,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",

)
bnb_4bit_compute_dtype="float16"
compute_dtype=getattr(torch,bnb_4bit_compute_dtype)

bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_quant_type="nf4",
    bnb_4it_use_double_quant=False
)

Unused kwargs: ['bnb_4it_use_double_quant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


In [ ]:
model=AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

In [ ]:
from trl import SFTConfig, SFTTrainer
training_args = SFTConfig(
    per_device_train_batch_size=5,
    gradient_accumulation_steps=10,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",


    max_steps=50,
    report_to="tensorboard",

    logging_steps=5,
    dataset_text_field="prompt",
    max_seq_length=1000,
    output_dir="./result",
    optim="paged_adamw_32bit",
    warmup_steps=25,
    bf16=True,





)

In [ ]:
from trl import SFTConfig, SFTTrainer
training_args = SFTConfig(
    max_seq_length=512,
    output_dir="/tmp",
)

In [ ]:
model.config.use_cache=False
model.config.pretraining_tp=1
model=get_peft_model(model, peft_config)
model=prepare_model_for_kbit_training(model,use_gradient_checkpointing=True)


In [ ]:
trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    args=training_args,
)

Tokenizing train dataset:   0%|          | 0/112560 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/112560 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
5,4.846600
10,4.882200
15,4.837200
20,4.865700
25,4.881400
30,4.887600
35,4.841700
40,4.840600
45,4.839500
50,4.857400


TrainOutput(global_step=50, training_loss=4.857978782653809, metrics={'train_runtime': 50.008, 'train_samples_per_second': 49.992, 'train_steps_per_second': 1.0, 'total_flos': 5121054720000.0, 'train_loss': 4.857978782653809})

In [ ]:
trainer.save_model("gpt2_finetuned_sft")
tokenizer.save_pretrained("gpt2_finetuned_tokenizer")

('gpt2_finetuned_tokenizer/tokenizer_config.json',
 'gpt2_finetuned_tokenizer/special_tokens_map.json',
 'gpt2_finetuned_tokenizer/vocab.json',
 'gpt2_finetuned_tokenizer/merges.txt',
 'gpt2_finetuned_tokenizer/added_tokens.json',
 'gpt2_finetuned_tokenizer/tokenizer.json')

In [ ]:


# Load the tokenizer from the saved directory
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/gpt2_finetuned_tokenizer")

# Similarly, reload the model
model = AutoModelForCausalLM.from_pretrained("/content/gpt2_finetuned_sft", trust_remote_code=True)



/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
import transformers
# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [ ]:
def chat(user):
    message = [
    {"role": "system", "content": "You are a helpful assistant chatbot."},
    {"role": "user", "content": user}
    ]
    prompt = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)
    sequences = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200

    )

    return sequences[0]['generated_text']

In [ ]:
def chat(user):
    message = [
    {"role": "system", "content": "You are a helpful assistant chatbot."},
    {"role": "user", "content": user}
    ]
    # Define a chat template string
    chat_template = """{% if messages[0]['role'] == 'system' %}{{messages[0]['content']}}{% endif %}{% for message in messages[1:] %}\n{{message['role']}}: {{message['content']}}{% endfor %}"""

    prompt = tokenizer.apply_chat_template(
        message,
        chat_template=chat_template,
        add_generation_prompt=True,
        tokenize=False
    )
    sequences = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200

    )

    return sequences[0]['generated_text']

In [ ]:
def chat(user):
    message = [
        {"role": "system", "content": "You are a helpful assistant chatbot."},
        {"role": "user", "content": user}
    ]
    # # Define a chat template string
    # chat_template = """{%if messages[0]['role'] == 'system' %}{{messages[0]['content']}}{% endif %}\nuser: {{messages[1]['content']}}"""
    chat_template = """{{bos}}User: {{user}}{{eos}}Assistant:"""
      # Simplified template


    prompt = tokenizer.apply_chat_template(
        message,
        chat_template=chat_template,
        add_generation_prompt=True,
        tokenize=False
    )
    sequences = pipeline(
        prompt,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        num_return_sequences=1,
        max_length=100
    )
    return sequences[0]['generated_text']

In [ ]:
user="What's the best thing about your daily routine?"
response=chat(user)


In [ ]:
from pprint import pprint

In [ ]:
response

'User: Assistant: Ms. Wyden\n\nName: Ms. Wyden\n\nAddress: 527 W. 8th St., Washington, DC 20003\n\nPhone: 202-664-3715\n\nEmail: wyden@senate.gov\n\nWebsite: http://www.senate.gov/en-us/press-releases/press-releases/press-releases-2011-01-28-en-us-press-re'

In [ ]:
pprint(response)

('User: Assistant: Ms. Wyden\n'
 '\n'
 'Name: Ms. Wyden\n'
 '\n'
 'Address: 527 W. 8th St., Washington, DC 20003\n'
 '\n'
 'Phone: 202-664-3715\n'
 '\n'
 'Email: wyden@senate.gov\n'
 '\n'
 'Website: '
 'http://www.senate.gov/en-us/press-releases/press-releases/press-releases-2011-01-28-en-us-press-re')


In [ ]:
user="Do you prefer spending time indoors or outdoors?"
response=chat(user)
pprint(response)

('User: Assistant: R.B.\n'
 '\n'
 'M: Assistant: G.C.\n'
 '\n'
 'L: Assistant: E.S.\n'
 '\n'
 'Q: What is the best way to get your name on the list?\n'
 '\n'
 'A: The most convenient way is to go to the website and download the name on '
 'the right hand side of the screen.\n'
 '\n'
 'Q: What is your favorite way to get your name on the list?\n'
 '\n'
 'A: You can always find a name on')
